### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import astropy.units as u
import numpy as np
from tabulate import tabulate
import tessellate
import simulate
from survey_tools import healpix

### Options

In [ ]:
config_filename = 'LGSAO.ini'
wavelength   = 1.65 * u.micron
zenith_angle = 30.0 * u.deg
seeing       = 0.60 * u.arcsec

### Compute Location of Points in FOV to Simulate

In [ ]:
inner_level = 14
fov_radius = 60.0 * u.arcsec
pix_area = healpix.get_area(inner_level)
num_pix_in_fov = int(np.ceil(np.pi * fov_radius**2 / pix_area))
num_rings = tessellate.closest_num_hex_rings(num_pix_in_fov)
num_points = tessellate.num_points_using_hex_rings(num_rings)
print(f"Num Points in {fov_radius.to(u.arcsec):.0f} FoV: {num_points}")

points, cells = tessellate.tessellate_circle_with_hexagons(num_rings, fov_radius.to(u.arcsec).value)

### Run Simulation

In [ ]:
results = simulate.run_simulation(config_filename, wavelength, zenith_angle, seeing, points=points, cells=cells, do_plot=True, verbose=True)

### Key Metrics

In [ ]:
print('Results:')
print(f"  Mean FWHM [mas]  : {np.mean(results.fwhm):.1f} ± {np.std(results.fwhm)/np.sqrt(len(results.fwhm)):.1f}")
print(f"  Mean Strehl Ratio: {np.mean(results.sr):.3f} ± {np.std(results.sr)/np.sqrt(len(results.sr)):.3f}")
print(f"  Mean EE [{results.ee_radius}mas] : {np.mean(results.ee):.3f} ± {np.std(results.ee)/np.sqrt(len(results.ee)):.3f}")

display(tabulate(np.column_stack((np.arange(len(points)), results.zd, results.az, results.fwhm, results.sr, results.ee)), headers=['Point', 'ZD [arcsec]', 'AZ [deg]', 'FWHM [mas]', 'SR', f"EE [{results.ee_radius}mas]"], floatfmt=['g', '.1f', '.1f', '.1f', '.2f', '.2f'], tablefmt='html'))

### Plot FOV

In [ ]:
simulate.plot_fov(results, bkg='FWHM', plot_fwhm_points=False)

# TODO:
# - read results from output fits instead of running sim which is very slow
# - use star magnitudes to calculate NumberPhotons i.e. detected flux in [ph/frame/subaperture]
